# Parameter tuning
## 1.Try different parameter on basic network structure， keep the best parameter
## 2. IF a word has a UpperCase character , it's very likely a skill word. 
##     Add a Case_sensitive layer to the network, test the accuracy

### import NER dataset

In [1]:
import pandas as pd
from pprint import pprint
from IPython.core.display import display, HTML

import warnings
warnings.filterwarnings('ignore')

In [2]:
import json
with open("NER_dataset/tran_X.json", "r") as f:
    X = json.load(f)
    
with open("NER_dataset/tran_Y.json", "r") as f:
    Y = json.load(f)

### sample data

In [3]:
print( X[0] )
print( Y[0] )

['Experience', 'in', 'commonly', 'used', 'for', 'data', 'analysis', 'such', 'as', 'Python', ',', 'R', ',', 'Julia', ',', 'or', 'SAS', '.']
['O', 'O', 'O', 'O', 'O', 'B', 'I', 'O', 'O', 'B', 'O', 'B', 'O', 'B', 'O', 'O', 'B', 'O']


In [4]:
print( 'trainset_size:',len( X ) )
print( 'testset_size:',len( Y ) )

trainset_size: 614
testset_size: 614


# dataset split

In [5]:
import numpy as np
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split( X , Y , test_size=0.2, random_state=42)

# PreProcess

In [6]:
import tensorflow as tf
from pprint import pprint

In [7]:
from keras.models import *
from keras.layers import *
from keras.optimizers import *
from keras.callbacks import *
from keras_contrib.layers import CRF
from keras_contrib.losses import crf_loss
from keras_contrib.metrics import crf_accuracy
from keras.preprocessing.sequence import pad_sequences
from keras_bert import load_trained_model_from_checkpoint
from keras_bert import Tokenizer
import matplotlib.pyplot as plt

Using TensorFlow backend.


In [8]:
max_seq_length = 128

In [8]:
label = {}
_label = {}


label_path = "uncased_L-2_H-128_A-2\tag_dict.txt"
f_label = open(label_path, 'r+', encoding='utf-8')
for line in f_label:
    content = line.strip().split()
    label[content[0].strip()] = content[1].strip()
    _label[content[1].strip()] = content[0].strip()
    
    
vocab = {}
vocab_path = r"uncased_L-2_H-128_A-2\vocab.txt"
with open(vocab_path, 'r+', encoding='utf-8') as f_vocab:
    for line in f_vocab.readlines():
        vocab[line.strip()] = len(vocab)

In [10]:
def PreProcessInputData( text ):
    word_labels = []
    seq_types = []

    for sequence in text:
        len_text = len(sequence)

        ###########################################
        temp_word_labels = []
        temp_word_labels.append( 101 )            
        for w in sequence:
            temp_word_labels.append( vocab.get(str(w).lower(),100) )
        temp_word_labels.append( 102 )

        ###########################################
        ###########################################
        temp_seq_types = [1] * len(temp_word_labels) +  [0] * ( max_seq_length - len( temp_word_labels ))
        temp_word_labels = temp_word_labels + [0] * ( max_seq_length - len( temp_word_labels ))

        word_labels.append( temp_word_labels )
        seq_types.append( temp_seq_types )

    return word_labels, seq_types

In [11]:
def PreProcessOutputData( text ):
    tags = []
    for line in text:
        tag = [0]
        for item in line:
            tag.append( int(label[item.strip()]) )
        tag.append(0)
        tags.append(tag)

    pad_tags = pad_sequences(tags, maxlen=max_seq_length, padding="post", truncating="post")
    result_tags = np.expand_dims(pad_tags, 2)
    return result_tags

# Data PreProcessing

In [12]:
train_sentence_list_word_labels ,train_sentence_list_seq_types = PreProcessInputData( X_train )
train_output_label_list = PreProcessOutputData( y_train )

In [13]:
test_sentence_list_word_labels ,test_sentence_list_seq_types = PreProcessInputData( X_test )
test_output_label_list = PreProcessOutputData( y_test )

# basic network structure define

# define_Layer

In [14]:
def define_Layer( layer_dict, my_input ):
    
    if layer_dict['layer_type'] == 'Bidirectional_LSTM':
        return Bidirectional( LSTM( layer_dict['lstmDim'],
                                         return_sequences = True,
                                         activation = layer_dict['activation'],
                                         dropout = layer_dict['dropout'],
                                         recurrent_dropout = layer_dict['dropout']) )(my_input)
    
    elif layer_dict['layer_type'] == 'Dense':
        return Dense(units = layer_dict['units'] , activation = layer_dict['activation'] )(my_input)
    
    elif layer_dict['layer_type'] == 'Dropout':
        return Dropout(dropout = layer_dict['dropout'] )(my_input)

# define network structure

In [15]:
def Train_Model( network_struct_dict ):
    print( '■' * 60 )
    print('network struct:')
    pprint( network_struct_dict )
    
    #################################################################
    ## use BERT embedding layer
    model_path = r"uncased_L-2_H-128_A-2/"
    bert = load_trained_model_from_checkpoint(
        model_path + "bert_config.json",
        model_path + "bert_model.ckpt",
        seq_len = max_seq_length
        )

    #make bert layer trainable
    for layer in bert.layers:
        layer.trainable = network_struct_dict['is_embeding_trainable']
        
    x1 = Input(shape=(None,))
    x2 = Input(shape=(None,))
    bert_out = bert([x1, x2])
    
    my_output = bert_out
    #################################################################
    # use layer_dict_List to build model
    for layer_dict in network_struct_dict['layer_dict_List']:
        my_input  = my_output
        my_output = define_Layer( layer_dict, my_input )
    
    
    #################################################################
    ## CRF output
    crf_out = CRF(len(label), sparse_target=True)(my_output)
    model = Model([x1, x2], crf_out)
    model.summary()

    model.compile(
        optimizer=network_struct_dict['optimizer'],
        loss=crf_loss,
        metrics=[crf_accuracy]
    )
    
    #################################################################
    ## training
    model.fit(x=[train_sentence_list_word_labels, train_sentence_list_seq_types],
                       y = train_output_label_list,
                       batch_size = network_struct_dict['batch_size'],
                       epochs = network_struct_dict['epochs'],
                       validation_split=0.2,
                       callbacks=[EarlyStopping(monitor='val_loss', patience=3, min_delta=0.001 )],
                       verbose = 1,
                       class_weight = 'auto')
    
    print('evaluate :')
    res = model.evaluate(x=[test_sentence_list_word_labels, test_sentence_list_seq_types],
                       y=test_output_label_list,batch_size=128)
    
    print( res )
    return res[1]

# try different parameter and keep the best

In [16]:
network_struct_dict_List = [
    ## network_struct
    {
        'is_embeding_trainable' : True,
        'optimizer' : RMSprop(1e-4),
        'batch_size' : 128,
        'epochs': 100,
        
        'layer_dict_List' : [ 
            {'layer_type':'Bidirectional_LSTM', 'lstmDim': 256 ,'activation':'tanh', 'dropout' :0.3},
        ]
    },
    
    ## network_struct
    {
        'is_embeding_trainable' : True,
        'optimizer' : RMSprop(1e-4),
        'batch_size' : 128,
        'epochs': 100,
        
        'layer_dict_List' : [ 
            {'layer_type':'Bidirectional_LSTM', 'lstmDim': 128 ,'activation':'tanh', 'dropout' :0.2},
        ]
    },
    
    ## network_struct
    {
        'is_embeding_trainable' : False,
        'optimizer' : Adam(1e-4),
        'batch_size' : 128,
        'epochs': 100,
        
        'layer_dict_List' : [ 
            {'layer_type':'Bidirectional_LSTM', 'lstmDim': 128 ,'activation':'tanh', 'dropout' :0.2},
        ]
    },
    
    ## network_struct
    {
        'is_embeding_trainable' : True,
        'optimizer' : Adam(1e-4),
        'batch_size' : 128,
        'epochs': 100,
        
        'layer_dict_List' : [ 
            {'layer_type':'Bidirectional_LSTM', 'lstmDim': 128 ,'activation':'sigmoid', 'dropout' :0.2},
        ]
    },
    
    ## network_struct
    {
        'is_embeding_trainable' : True,
        'optimizer' : RMSprop(1e-4),
        'batch_size' : 128,
        'epochs': 100,
        
        'layer_dict_List' : [ 
            {'layer_type':'Bidirectional_LSTM', 'lstmDim': 64 ,'activation':'tanh', 'dropout' :0.2},
        ]
    },
    
    
    ## network_struct
    {
        'is_embeding_trainable' : True,
        'optimizer' : RMSprop(1e-4),
        'batch_size' : 128,
        'epochs': 100,
        
        'layer_dict_List' : [ 
            {'layer_type':'Bidirectional_LSTM', 'lstmDim': 128 ,'activation':'tanh', 'dropout' :0.4},
        ]
    },
    
    ## network_struct
    {
        'is_embeding_trainable' : True,
        'optimizer' : RMSprop(1e-4),
        'batch_size' : 128,
        'epochs': 100,
        
        'layer_dict_List' : [ 
            {'layer_type':'Bidirectional_LSTM', 'lstmDim': 256 ,'activation':'tanh', 'dropout' :0.2},
            {'layer_type':'Bidirectional_LSTM', 'lstmDim': 128 ,'activation':'tanh', 'dropout' :0.2},
        ]
    },
    
    ## network_struct
    {
        'is_embeding_trainable' : True,
        'optimizer' : RMSprop(1e-4),
        'batch_size' : 128,
        'epochs': 100,
        
        'layer_dict_List' : [ 
            {'layer_type':'Bidirectional_LSTM', 'lstmDim': 128 ,'activation':'tanh', 'dropout' :0.2},
            {'layer_type':'Bidirectional_LSTM', 'lstmDim': 64 ,'activation':'tanh', 'dropout' :0.2},
        ]
    },
    
    ## network_struct
    {
        'is_embeding_trainable' : True,
        'optimizer' : RMSprop(1e-4),
        'batch_size' : 128,
        'epochs': 100,
        
        'layer_dict_List' : [ 
            {'layer_type':'Bidirectional_LSTM', 'lstmDim': 128 ,'activation':'tanh', 'dropout' :0.2},
            {'layer_type':'Bidirectional_LSTM', 'lstmDim': 64 ,'activation':'sigmoid', 'dropout' :0.2},
            {'layer_type':'Bidirectional_LSTM', 'lstmDim': 32 ,'activation':'sigmoid', 'dropout' :0.2},
        ]
    },
    
    
    ## network_struct
    {
        'is_embeding_trainable' : True,
        'optimizer' : RMSprop(1e-4),
        'batch_size' : 64,
        'epochs': 100,
        'layer_dict_List' : [ 
            {'layer_type':'Bidirectional_LSTM', 'lstmDim': 256 ,'activation':'tanh', 'dropout' :0.3},
            {'layer_type':'Dense', 'units': 64 ,'activation':'sigmoid'},
        ]
    },
    
    ## network_struct
    {
        'is_embeding_trainable' : True,
        'optimizer' : Adam(1e-4),
        'batch_size' : 64,
        'epochs': 100,
        
        'layer_dict_List' : [ 
            {'layer_type':'Bidirectional_LSTM', 'lstmDim': 128 ,'activation':'tanh', 'dropout' :0.3},
            {'layer_type':'Dense', 'units': 32 ,'activation':'sigmoid'},
        ]
    }
]

In [17]:
best_network_struct = None
best_accurac = 0

for network_struct_dict in network_struct_dict_List:
    train_accurac = Train_Model( network_struct_dict )
    if best_accurac < train_accurac:
        best_accurac = train_accurac
        best_network_struct = network_struct_dict
        
print('Best Network Struct:')
pprint(best_network_struct)

■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■
network struct:
{'batch_size': 128,
 'epochs': 100,
 'is_embeding_trainable': True,
 'layer_dict_List': [{'activation': 'tanh',
                      'dropout': 0.3,
                      'layer_type': 'Bidirectional_LSTM',
                      'lstmDim': 256}],
 'optimizer': <keras.optimizers.RMSprop object at 0x00000000196F1320>}




Instructions for updating:
Please use `rate` instead of `keep_prob`. Rate should be set to `rate = 1 - keep_prob`.







Instructions for updating:
Use tf.where in 2.0, which has the same broadcast rule as np.where
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_1 (InputLayer)            (None, None)         0                                            
_______________________________________________________________________________________

Epoch 19/100
392/392 [==============================] - 19s 49ms/step - loss: 11.5261 - crf_accuracy: 0.8719 - val_loss: 11.1614 - val_crf_accuracy: 0.8891
Epoch 20/100
392/392 [==============================] - 20s 50ms/step - loss: 11.5092 - crf_accuracy: 0.8783 - val_loss: 11.1592 - val_crf_accuracy: 0.8831
Epoch 21/100
392/392 [==============================] - 19s 49ms/step - loss: 11.4914 - crf_accuracy: 0.8908 - val_loss: 11.1449 - val_crf_accuracy: 0.8937
Epoch 22/100
392/392 [==============================] - 21s 53ms/step - loss: 11.4858 - crf_accuracy: 0.8880 - val_loss: 11.1471 - val_crf_accuracy: 0.8992
Epoch 23/100
392/392 [==============================] - 20s 50ms/step - loss: 11.4829 - crf_accuracy: 0.8882 - val_loss: 11.1666 - val_crf_accuracy: 0.8767
Epoch 24/100
392/392 [==============================] - 20s 50ms/step - loss: 11.4697 - crf_accuracy: 0.8948 - val_loss: 11.1402 - val_crf_accuracy: 0.8946
Epoch 25/100
392/392 [==============================] - 20s 51ms

392/392 [==============================] - 19s 48ms/step - loss: 11.4543 - crf_accuracy: 0.8978 - val_loss: 11.2245 - val_crf_accuracy: 0.8456
Epoch 27/100
392/392 [==============================] - 18s 47ms/step - loss: 11.4711 - crf_accuracy: 0.8888 - val_loss: 11.1433 - val_crf_accuracy: 0.8914
Epoch 28/100
392/392 [==============================] - 19s 48ms/step - loss: 11.4459 - crf_accuracy: 0.9009 - val_loss: 11.1342 - val_crf_accuracy: 0.9065
Epoch 29/100
392/392 [==============================] - 19s 48ms/step - loss: 11.4341 - crf_accuracy: 0.9062 - val_loss: 11.1579 - val_crf_accuracy: 0.8804
Epoch 30/100
392/392 [==============================] - 19s 47ms/step - loss: 11.4297 - crf_accuracy: 0.9071 - val_loss: 11.1650 - val_crf_accuracy: 0.8721
Epoch 31/100
392/392 [==============================] - 19s 47ms/step - loss: 11.4286 - crf_accuracy: 0.9085 - val_loss: 11.1383 - val_crf_accuracy: 0.8914
evaluate :
123/123 [==============================] - 1s 11ms/step
[11.307711

392/392 [==============================] - 15s 37ms/step - loss: 11.6737 - crf_accuracy: 0.7978 - val_loss: 11.2829 - val_crf_accuracy: 0.8341
Epoch 87/100
392/392 [==============================] - 14s 37ms/step - loss: 11.6732 - crf_accuracy: 0.7996 - val_loss: 11.2799 - val_crf_accuracy: 0.8341
Epoch 88/100
392/392 [==============================] - 14s 37ms/step - loss: 11.6606 - crf_accuracy: 0.8043 - val_loss: 11.2759 - val_crf_accuracy: 0.8355
Epoch 89/100
392/392 [==============================] - 14s 37ms/step - loss: 11.6621 - crf_accuracy: 0.8005 - val_loss: 11.2724 - val_crf_accuracy: 0.8364
Epoch 90/100
392/392 [==============================] - 14s 36ms/step - loss: 11.6599 - crf_accuracy: 0.8023 - val_loss: 11.2699 - val_crf_accuracy: 0.8359
Epoch 91/100
392/392 [==============================] - 14s 37ms/step - loss: 11.6543 - crf_accuracy: 0.8081 - val_loss: 11.2673 - val_crf_accuracy: 0.8401
Epoch 92/100
392/392 [==============================] - 14s 37ms/step - loss:

392/392 [==============================] - 16s 40ms/step - loss: 11.6705 - crf_accuracy: 0.7994 - val_loss: 11.3101 - val_crf_accuracy: 0.8295
Epoch 24/100
392/392 [==============================] - 16s 40ms/step - loss: 11.6641 - crf_accuracy: 0.8059 - val_loss: 11.3197 - val_crf_accuracy: 0.8130
Epoch 25/100
392/392 [==============================] - 16s 40ms/step - loss: 11.6418 - crf_accuracy: 0.8107 - val_loss: 11.2897 - val_crf_accuracy: 0.8359
Epoch 26/100
392/392 [==============================] - 16s 40ms/step - loss: 11.6346 - crf_accuracy: 0.8244 - val_loss: 11.2879 - val_crf_accuracy: 0.8281
Epoch 27/100
392/392 [==============================] - 16s 40ms/step - loss: 11.6226 - crf_accuracy: 0.8263 - val_loss: 11.2810 - val_crf_accuracy: 0.8300
Epoch 28/100
392/392 [==============================] - 16s 40ms/step - loss: 11.6091 - crf_accuracy: 0.8310 - val_loss: 11.2670 - val_crf_accuracy: 0.8428
Epoch 29/100
392/392 [==============================] - 16s 40ms/step - loss:

...

Train on 392 samples, validate on 99 samples
Epoch 1/100
392/392 [==============================] - 91s 233ms/step - loss: 13.2155 - crf_accuracy: 0.0000e+00 - val_loss: 12.7754 - val_crf_accuracy: 0.0000e+00
Epoch 2/100
392/392 [==============================] - 68s 175ms/step - loss: 13.0794 - crf_accuracy: 0.0000e+00 - val_loss: 12.6719 - val_crf_accuracy: 0.0000e+00
Epoch 3/100
392/392 [==============================] - 71s 182ms/step - loss: 12.9848 - crf_accuracy: 0.0000e+00 - val_loss: 12.5833 - val_crf_accuracy: 0.0000e+00
Epoch 4/100
392/392 [==============================] - 67s 172ms/step - loss: 12.9017 - crf_accuracy: 0.0027 - val_loss: 12.5001 - val_crf_accuracy: 0.0151
Epoch 5/100
392/392 [==============================] - 69s 177ms/step - loss: 12.8244 - crf_accuracy: 0.0201 - val_loss: 12.4234 - val_crf_accuracy: 0.0555
Epoch 6/100
392/392 [==============================] - 67s 172ms/step - loss: 12.7530 - crf_accuracy: 0.0483 - val_loss: 12.3532 - val_crf_accuracy: 0.

Epoch 53/100
392/392 [==============================] - 65s 166ms/step - loss: 11.8360 - crf_accuracy: 0.7310 - val_loss: 11.4835 - val_crf_accuracy: 0.7333
Epoch 54/100
392/392 [==============================] - 66s 168ms/step - loss: 11.8214 - crf_accuracy: 0.7321 - val_loss: 11.4890 - val_crf_accuracy: 0.7333
Epoch 55/100
392/392 [==============================] - 67s 171ms/step - loss: 11.8172 - crf_accuracy: 0.7319 - val_loss: 11.4753 - val_crf_accuracy: 0.7379
Epoch 56/100
392/392 [==============================] - 66s 167ms/step - loss: 11.7989 - crf_accuracy: 0.7345 - val_loss: 11.4511 - val_crf_accuracy: 0.7447
Epoch 57/100
392/392 [==============================] - 65s 166ms/step - loss: 11.7855 - crf_accuracy: 0.7420 - val_loss: 11.4450 - val_crf_accuracy: 0.7475
Epoch 58/100
392/392 [==============================] - 65s 166ms/step - loss: 11.7767 - crf_accuracy: 0.7412 - val_loss: 11.4332 - val_crf_accuracy: 0.7759
Epoch 59/100
392/392 [==============================] - 65

Train on 392 samples, validate on 99 samples
Epoch 1/100
392/392 [==============================] - 88s 224ms/step - loss: 12.7742 - crf_accuracy: 0.2830 - val_loss: 11.9236 - val_crf_accuracy: 0.6885
Epoch 2/100
392/392 [==============================] - 77s 198ms/step - loss: 12.2521 - crf_accuracy: 0.6934 - val_loss: 11.7651 - val_crf_accuracy: 0.7340
Epoch 3/100
392/392 [==============================] - 74s 188ms/step - loss: 12.1230 - crf_accuracy: 0.7281 - val_loss: 11.7183 - val_crf_accuracy: 0.7340
Epoch 4/100
392/392 [==============================] - 73s 187ms/step - loss: 12.0745 - crf_accuracy: 0.7301 - val_loss: 11.6923 - val_crf_accuracy: 0.7340
Epoch 5/100
392/392 [==============================] - 74s 189ms/step - loss: 12.0436 - crf_accuracy: 0.7303 - val_loss: 11.6771 - val_crf_accuracy: 0.7340
Epoch 6/100
392/392 [==============================] - 75s 192ms/step - loss: 12.0156 - crf_accuracy: 0.7307 - val_loss: 11.6277 - val_crf_accuracy: 0.7340
Epoch 7/100
392/392

Train on 392 samples, validate on 99 samples
Epoch 1/100
392/392 [==============================] - 82s 208ms/step - loss: 13.0438 - crf_accuracy: 0.0547 - val_loss: 12.4297 - val_crf_accuracy: 0.2481
Epoch 2/100
392/392 [==============================] - 67s 170ms/step - loss: 12.7278 - crf_accuracy: 0.3751 - val_loss: 12.1843 - val_crf_accuracy: 0.7340
Epoch 3/100
392/392 [==============================] - 67s 170ms/step - loss: 12.5280 - crf_accuracy: 0.7256 - val_loss: 12.0497 - val_crf_accuracy: 0.7340
Epoch 4/100
392/392 [==============================] - 66s 168ms/step - loss: 12.4109 - crf_accuracy: 0.7313 - val_loss: 11.9722 - val_crf_accuracy: 0.7340
Epoch 5/100
392/392 [==============================] - 66s 168ms/step - loss: 12.3357 - crf_accuracy: 0.7309 - val_loss: 11.9256 - val_crf_accuracy: 0.7340
Epoch 6/100
392/392 [==============================] - 65s 167ms/step - loss: 12.2860 - crf_accuracy: 0.7301 - val_loss: 11.8942 - val_crf_accuracy: 0.7340
Epoch 7/100
392/392

Epoch 53/100
392/392 [==============================] - 67s 171ms/step - loss: 11.5542 - crf_accuracy: 0.9205 - val_loss: 11.3378 - val_crf_accuracy: 0.8739
Epoch 54/100
392/392 [==============================] - 68s 173ms/step - loss: 11.5407 - crf_accuracy: 0.9261 - val_loss: 11.3370 - val_crf_accuracy: 0.8933
evaluate :
123/123 [==============================] - 4s 33ms/step
[11.494059562683105, 0.8901559114456177]
Best Network Struct:
{'batch_size': 128,
 'epochs': 100,
 'is_embeding_trainable': True,
 'layer_dict_List': [{'activation': 'tanh',
                      'dropout': 0.2,
                      'layer_type': 'Bidirectional_LSTM',
                      'lstmDim': 256},
                     {'activation': 'tanh',
                      'dropout': 0.2,
                      'layer_type': 'Bidirectional_LSTM',
                      'lstmDim': 128}],
 'optimizer': <keras.optimizers.RMSprop object at 0x0000000019126AC8>}


# ■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■

## IF a word has a uppercase character ，it’s very likely a skill word

## change network structure ,add a addtional input layer
## change dataset if word contains uppercase label 1， if word are lowercase label 0
## test result

In [18]:
def PreProcessInputData_Addtion( text ):
    word_labels = []
    seq_types = []
    upper_list = []
    

    for sequence in text:
        len_text = len(sequence)

        ###########################################
        temp_word_labels = []
        temp_upper_list = []
        
        temp_word_labels.append( 101 )
        temp_upper_list.append(0)
        for w in sequence:
            temp_word_labels.append( vocab.get(str(w).lower(),100) )
            if str(w).isalpha() and (str(w).islower() == False):
                temp_upper_list.append( 2 )
            else:
                temp_upper_list.append( 1 )
            
        temp_word_labels.append( 102 )
        temp_upper_list.append(0)
        ###########################################
        ###########################################
        temp_seq_types = [1] * len(temp_word_labels) +  [0] * ( max_seq_length - len( temp_word_labels ))
        temp_word_labels = temp_word_labels + [0] * ( max_seq_length - len( temp_word_labels ))
        temp_upper_list  = temp_upper_list  + [0] * ( max_seq_length - len( temp_upper_list ))

        word_labels.append( temp_word_labels )
        seq_types.append( temp_seq_types )
        upper_list.append( temp_upper_list )

    return word_labels, seq_types, upper_list

In [19]:
def PreProcessOutputData_Addtion( text ):
    tags = []
    for line in text:
        tag = [0]
        for item in line:
            tag.append( int(label[item.strip()]) )
        tag.append(0)
        tags.append(tag)

    pad_tags = pad_sequences(tags, maxlen=max_seq_length, padding="post", truncating="post")
    result_tags = np.expand_dims(pad_tags, 2)
    return result_tags

In [20]:
train_sentence_list_word_labels ,train_sentence_list_seq_types , train_sentence_upper_list = PreProcessInputData_Addtion( X_train )
train_output_label_list = PreProcessOutputData_Addtion( y_train )

In [21]:
test_sentence_list_word_labels ,test_sentence_list_seq_types , test_sentence_upper_list = PreProcessInputData_Addtion( X_test )
test_output_label_list = PreProcessOutputData_Addtion( y_test )

In [22]:
def Train_Model_Addtion( network_struct_dict ):
    print( '■' * 60 )
    print('network struct:')
    pprint( network_struct_dict )
    
    #################################################################
    ## use BERT embedding layer
    model_path = r"uncased_L-2_H-128_A-2/"
    bert = load_trained_model_from_checkpoint(
        model_path + "bert_config.json",
        model_path + "bert_model.ckpt",
        seq_len = max_seq_length
        )

    #make bert layer trainable
    for layer in bert.layers:
        layer.trainable = network_struct_dict['is_embeding_trainable']
        
    x1 = Input(shape=(None,))
    x2 = Input(shape=(None,))
    x3 = Input(shape=(None,))
    
    bert_out = bert([x1, x2])
    label_layer1 = Embedding(4, 2, trainable=False)(x3)

    x4 = Concatenate()( [bert_out, label_layer1] )
    print(x4.shape)
    
    my_output = x4
    #################################################################
    # use layer_dict_List to build model
    for layer_dict in network_struct_dict['layer_dict_List']:
        my_input  = my_output
        my_output = define_Layer( layer_dict, my_input )
    
    
    #################################################################
    ## CRF output
    crf_out = CRF(len(label), sparse_target=True)(my_output)
    model = Model([x1, x2, x3], crf_out)
    model.summary()

    model.compile(
        optimizer=network_struct_dict['optimizer'],
        loss=crf_loss,
        metrics=[crf_accuracy]
    )
    
    #################################################################
    ## training
    model.fit( [train_sentence_list_word_labels, train_sentence_list_seq_types, train_sentence_upper_list],
                       y = train_output_label_list,
                       batch_size = network_struct_dict['batch_size'],
                       epochs = network_struct_dict['epochs'],
                       validation_split=0.2,
                       callbacks=[EarlyStopping(monitor='val_loss', patience=3, min_delta=0.001)],
                       verbose = 1,
                       class_weight = 'auto')
    
    print('evaluate :')
    res = model.evaluate( [test_sentence_list_word_labels, test_sentence_list_seq_types, test_sentence_upper_list],
                       y=test_output_label_list,batch_size=128)
    
    print( res )
    return res[1]

In [27]:
network_struct_dict = {
    'is_embeding_trainable' : True,
    'optimizer' : Adam(1e-4),
    'batch_size' : 32,
    'epochs': 100,

    'layer_dict_List' : [ 
        {'layer_type':'Bidirectional_LSTM', 'lstmDim': 256 ,'activation':'tanh', 'dropout' :0.3},
#         {'layer_type':'Bidirectional_LSTM', 'lstmDim': 128 ,'activation':'tanh', 'dropout' :0.2},
    ]
}

In [24]:
best_network_struct

{'is_embeding_trainable': True,
 'optimizer': <keras.optimizers.RMSprop at 0x19126ac8>,
 'batch_size': 128,
 'epochs': 100,
 'layer_dict_List': [{'layer_type': 'Bidirectional_LSTM',
   'lstmDim': 256,
   'activation': 'tanh',
   'dropout': 0.2},
  {'layer_type': 'Bidirectional_LSTM',
   'lstmDim': 128,
   'activation': 'tanh',
   'dropout': 0.2}]}

In [28]:
Train_Model_Addtion( network_struct_dict )

■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■
network struct:
{'batch_size': 32,
 'epochs': 100,
 'is_embeding_trainable': True,
 'layer_dict_List': [{'activation': 'tanh',
                      'dropout': 0.3,
                      'layer_type': 'Bidirectional_LSTM',
                      'lstmDim': 256}],
 'optimizer': <keras.optimizers.Adam object at 0x000000006A262D68>}
(?, ?, 130)
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_26 (InputLayer)           (None, None)         0                                            
__________________________________________________________________________________________________
input_27 (InputLayer)           (None, None)         0                                            
__________________________________________________________________________________________________
in

0.9106805324554443